In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.measure import compare_ssim
from os import listdir, path
from tempfile import TemporaryFile
import pymysql

%matplotlib inline

In [2]:
import pymysql


def connect_db(sql,action):
    
    r=[]
    if action == "u":
        cursor.execute(sql)
        db.commit()
    if action == "s":
        cursor.execute(sql)
        results = cursor.fetchall()
        for row in results:
            r.append(row)
        return r

db = pymysql.connect("localhost","Sebrm","sebrm","Rumbomobile")
cursor = db.cursor()


In [3]:
def obtain_pklots(entry,coords,coordsX,coordsY):
    masks = []
    image = entry
    for i in range(len(coords)):
        ROI_corners = np.array([coords[i].tolist()], dtype=np.int32) 
        mask = np.zeros(image.shape, dtype=np.uint8)
        cv2.fillPoly(mask, ROI_corners, (255))
        masked_image = cv2.bitwise_and(image, mask)
        masked_image = masked_image[np.min(coordsY[i]):np.max(coordsY[i]),np.min(coordsX[i]):np.max(coordsX[i])]
        masks.append(masked_image)
    return masks

In [4]:
def read_images(path):
    img_list   = {}
    img_clist  = {}
    files_imgs = sorted(listdir(path))
    cont       = 0
#     print(files_imgs)
    for i in files_imgs:
        img_clist[str(cont)] = cv2.imread(path+'/'+i,cv2.IMREAD_COLOR)
        img_list[str(cont)] = cv2.imread(path+'/'+i,cv2.IMREAD_GRAYSCALE)
        cont = cont + 1
    
    return [img_list,img_clist]

In [5]:
def compare(pklot_empty,pklot_ref,pklot_new,state_old):
    scores     = []
    scores_emp = []
    bck_diff   = []
    stat       = []
    for i in range(len(pklot_ref)): 
        
        img_emp = pklot_empty[i]
        img_A   = pklot_ref[i]
        img_B   = pklot_new[i]

        bck_diff.append(img_B-img_A)

        
        (score_emp,_) = compare_ssim(img_emp, img_B, full=True)
        (score,_)     = compare_ssim(img_A, img_B, full=True)
        score         = np.float32(score)
        
        scores.append(score)
        scores_emp.append(score_emp)
        
        if score > 0.65 :
            stat.append(state_old[i])
        else:
            
            if state_old[i] == "occupied":
                if score_emp < 0.6:
                    stat.append("occupied")
                else:
                    stat.append("empty")
            else:
                stat.append("occupied")
                
    
    
    return [scores_emp,scores,stat]

In [47]:
def show_pklots(img_color,coords,state):
    
    
    for i in range(len(coords)):
        if state[i] == "empty":
            color = (0,255,0)
        else:
            color = (0,0,255)
#         cv2.line(img_color, coords[i][3], coords[i][0], color, thickness=2, lineType=8, shift=0)
#         cv2.line(img_color, coords[i][0], coords[i][1], color, thickness=2, lineType=8, shift=0)
#         cv2.line(img_color, coords[i][1], coords[i][2], color, thickness=2, lineType=8, shift=0)
#         cv2.line(img_color, coords[i][2], coords[i][3], color, thickness=2, lineType=8, shift=0)
        font = cv2.FONT_HERSHEY_SIMPLEX

        cv2.putText(img_color,str(i),coords[i][0], font, 1,color,2,cv2.LINE_AA)

#     plt.figure(figsize=(16,16))
#     plt.imshow(img_color)
#     plt.show()
    return img_color

In [8]:
sql = "SELECT * FROM virtualpklot1"
data = connect_db(sql,"s")
len(data)

72

In [9]:
coordsX=[]
b=[coordsX.append(data[i][1].split(",")) for i in range(len(data))]
coordsY=[]
b=[coordsY.append(data[i][2].split(",")) for i in range(len(data))]

coordsX = np.float32(coordsX)
coordsY = np.float32(coordsY)
coords = np.zeros(coordsX.shape,dtype = tuple)
coords_mag = np.sqrt(coordsX**2 + coordsY**2)
coords_mag.sort()
minmag = coords_mag[:,0]
maxmag = coords_mag[:,3]

for i in range(len(coordsX)):
    k = 1
    for j in range(len(coordsX[1])):
        
        if (np.abs(np.sqrt(coordsX[i][j]**2 + coordsY[i][j]**2) - minmag[i]) <=0.01):
            coords[i][0]=(coordsX[i][j],coordsY[i][j])
        elif (np.abs(np.sqrt(coordsX[i][j]**2 + coordsY[i][j]**2) - maxmag[i]) <=0.01):
            coords[i][2]=(coordsX[i][j],coordsY[i][j])
        else:
            coords[i][k]=(coordsX[i][j],coordsY[i][j])
            k = k + 2
        
            
coords.tolist();
coordsX = np.uint16(coordsX)
coordsY = np.uint16(coordsY)

In [76]:
def detect(img,ind,pklot_old,state):
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    pklot_new = obtain_pklots(gray,coords,coordsX,coordsY)
    comp_e,comp,temp = compare(pklot_empty,pklot_old,pklot_new,state[ind])
    state = temp
    
    return state,pklot_new

In [77]:
init = cv2.imread('alpha_test/Moment_0.jpg',cv2.IMREAD_GRAYSCALE)
c_state=[]
pklot_empty = obtain_pklots(init,coords,coordsX,coordsY)
pklot_new = obtain_pklots(init,coords,coordsX,coordsY)
state = []
comp = []
ms = []
bck_diff = []
for i in range(len(pklot_empty)): 

    img_A = pklot_empty[i]
    img_B = pklot_new[i]
    
    bck_diff.append(img_B-img_A)
    
    (score, diff) = compare_ssim(img_A, img_B, full=True)
    comp.append(score)
    
comp = np.float32(comp)
# print(compare)
meh2= (comp>0.5)
# print(compare)
# print(meh2)
old_state = []
for i in range(len(meh2)):
    if meh2[i]:
        old_state.append("empty")
    else:
        old_state.append("occupied")
        
pklot_old = pklot_new
c_state.append(old_state)
old_state = c_state

In [78]:
cap = cv2.VideoCapture('alpha_test/pklotest.mp4')
ind = 0

fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (1920,1080))

while(cap.isOpened()):
    ret, frame = cap.read()
    
    c_state,pklot = detect(frame,ind,pklot_old,old_state)
    
    p = show_pklots(frame,coords,c_state)
    
    pklot_old = pklot
    old_state.append(c_state)
    out.write(p)
    cv2.imshow('frame',p)
    ind = ind + 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [79]:
len(old_state)

1542

In [29]:
len(coords)

72

In [30]:
len(c_state)

1

In [51]:
len(old_state)

694

In [7]:
cap = cv2.VideoCapture('alpha_test/pklotest.mp4')
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))

while(cap.isOpened()):
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

2195.0


In [ ]:
cap.get(CV_CAP_PROP_FRAME_COUNT)